# Trabajo práctico integrador

## Introducción
Una variable aleatoria es una función que transforma un conjunto de números aleatorios en una sucesión que se corresponde con una distribución de probabilidad. Por lo tanto, la eficacia de este generador está condicionada por la calidad del generador de números aleatorios. No solo se trata de que los números aleatorios sean probabilísticamente de distribución uniforme e independientes, sino que el generador tenga un ciclo suficientemente amplio para obtener resultados verosímiles, sin que esto signifique que genere un impacto importante en el rendimiento del sistema. Por lo cual, en el siguiente informe nos proponemos analizar dicha influencia de los números aleatorios en la generación de variables aleatorias a partir de estos, desarrollando cada método desde cero y comparando los resultados de múltiples situaciones distintas.

### Objetivos del proyecto
- Desarrollar generadores de variables aleatorias eficaces que cubran un amplio espectro de distribuciones de probabilidad
- Probar la dependencia de estos de la calidad del generador de números aleatorios
- Desarrollar algoritmos que siempre generen variables aleatorias de calidad y de bajo impacto en el sistema

### Alcances del proyecto
El proyecto será desarrollado en su totalidad en Python. Se pueden distinguir las etapas expuestas a continuación.

1. **Desarrollo de generadores de números aleatorios:** Estos se desarrollarán de forma totalmente manual para tener total control de la calidad de los mismos. Se crearán generadores tanto congruenciales lineales como de cuadrados medios. De los primeros se brindará la opción de obtener generadores multiplicativos y mixtos y métodos para probar si son de ciclo completo.

2. **Validación de generadores de números aleatorios:** Se desarrollarán pruebas para validar la uniformidad e independencia estadísticas de las sucesiones de números obtenidas con los generadores antes dichos.

3. **Desarrollo de generadores de variables aleatorias:** Estos transformarán la sucesión de un generador especificado en una sucesión de variables aleatorias según una distribución a elección.

4. **Validación de generadores de variables aleatorias:** Se probarán los generadores de variables aleatorias con distintas sucesiones de números aleatorios para analizar el impacto de la calidad del primero en el segundo. Para esto, se generarán gráficas haciendo uso de la librería matplotlib.

# Implementación en Python

A continuación se explicarán los generadores de números pseudoaleatorios, los generadores de variables aleatorias y las pruebas de aletoriedad implementados en Python.

## Generador de números pseudoaleatorios

Para facilitar la creación de generadores de números pseudoaleatorios a lo largo del trabajo práctico, se decidió abstraer el concepto de *generador de números aleatorios* con la clase `Generator`. Todos los generadores son subclases de `Generator`, y deben implementar el método `next()`, el cual retorna el siguiente número aleatorio. También debe implementar el método `get_random_numbers()` y el método `get_xn_sequence()`, los cuales devuelve la secuencia entera de números pseudoaleatorios $\{\mu_0, \mu_1, \dots, \mu_n\}$ y de $x_n$ $\{x_0, x_1, \dots, x_n\}$ generados. Por último deben implementar el método `verify_parameters()`, que verifica la validez de los parámetros del generador.

Además, todas las subclases de `Generador` cuentan con el método `plot_random_numbers()`, que grafica la secuencia de números pseudoaleatorios en el plano $xy$, donde el eje $x$ representa el índice $i$ del número pseudoaleatorio, y el eje $y$ representa el valor del número psuedoaleatorio $\mu_i$. Éste método admite el parámetro `join_points`, que en el caso de ser verdadero, une los puntos entre los números pseudoaleatorios, facilitando la visualizacion del orden de la secuencia $\mu_1, \mu_2, \dots, \mu_n$ generada.

#### Ejemplos

##### Ejemplo: Creación de generador congruencial mixto
Para poder probar los distintos métodos, se presenta un ejemplo de generador. Estos serán explicados en detalle más adelante.

In [2]:
from src.random_number import MixedCongruentialGenerator

generator = MixedCongruentialGenerator(seed=8, a=4, b=7, m=9)

##### Ejemplo: Obtención de números aleatorios enteros
El método `get_xn_sequence()` permite obtener una lista de números aleatorios iguales o mayores que cero. La longitud de la lista es el período máximo que permite el generador sin que ningún número se repita.

In [4]:
generator.get_xn_sequence()

[8, 3, 1, 2, 6, 4, 5, 0, 7]

##### Ejemplo: Obtención de números aleatorios
El método `get_random_numbers()` retorna una lista de números reales que oscilan entre 0 y 1. De la misma manera que el anterior, devuelve el período maximal completo sin repetir ningún número.

In [5]:
generator.get_random_numbers()

[0.8888888888888888,
 0.3333333333333333,
 0.1111111111111111,
 0.2222222222222222,
 0.6666666666666666,
 0.4444444444444444,
 0.5555555555555556,
 0.0,
 0.7777777777777778]

##### Ejemplo: Obtención de un único número aleatorio
El método `next()` permite obtenre el número aleatorio siguiente al último obtenido.

In [6]:
generator.next()

0.3333333333333333

### Generador congruencial lineal
`LinearCongruentialGenerator` es una clase abstracta que representa el concepto de *generador congruencial lineal*. Se concreta en dos generadores, a saber, el `MultiplicativeCongruentialGenerator` y el `MixedCongruentialGenerator`.

Los generadores congruenciales lineales se definen denotando la relación recursiva de la siguiente manera: 

$X_{n+1} = \left( a X_n + b \right)\bmod m$

donde $X$ es la secuencia de valores pseudoaleatorios objetivo ($x_0$ es la semilla), $m$ es el módulo, $a$ el elemento multiplicativo y $b$ es el elemento aditivo.

La clase abstracta cuenta con un único método a implementar obligatoriamente: `has_max_sequence()`, que devuelve un booleano que indica si el generador es de ciclo completo. Por su parte, implementa los métodos que hereda de `Generator` y lanza un error si se intenta crear un *Generador* con parámetros inválidos. Por ejemplo, impide que se asigne al elemento multiplicativo un número superior al módulo.

##### Ejemplo: Generador congruencial mixto

In [15]:
from src.random_number import MixedCongruentialGenerator

generator = MixedCongruentialGenerator(seed=8,a=4,b=7,m=9)
generator.get_xn_sequence()

[8, 3, 1, 2, 6, 4, 5, 0, 7]

##### Ejemplo: Generador congruencial multiplicativo

In [13]:
from src.random_number import MultiplicativeCongruentialGenerator

generator = MultiplicativeCongruentialGenerator(seed=8,a=4,m=9)
generator.get_xn_sequence()

[8, 5, 2]

##### Ejemplo: Verificación de si es de ciclo completo

In [18]:
generator.has_max_sequence()

True

### Generador de cuadrados medios

Los generadores de cuadrados medios serán instancias de la clase `MiddleSquareGenerator`. Su constructor recibe los parámetros:

$k > 0 $ - es el número de dígitos de los números aleatorios resultantes

$seed > 0$ - es la semilla o valor inicial

##### Ejemplo: Generador de cuadrados medios

In [3]:
from src.random_number import MiddleSquareGenerator

generator = MiddleSquareGenerator(k=2, seed=84)
generator.get_xn_sequence()

[84, 87, 89, 42, 11, 64, 77, 53, 90, 10, 0]